In [2]:
import os 
import numpy as np
import math
import pandas as pd
import itertools
import torch
from torch import nn, Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from typing import Optional, Union, Tuple
from pathlib import Path
import positional_encoder as pe
import matplotlib.pyplot as plt

# Utils

In [3]:
def generate_square_subsequent_mask(dim1: int, dim2: int, device: Optional[torch.device] = None) -> Tensor:
    """
    Generates an upper-triangular matrix of -inf, with zeros on diag.
    Modified from: 
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html

    Args:
        dim1: int, for both src and tgt masking, this must be target sequence
              length
        dim2: int, for src masking this must be encoder sequence length (i.e. 
              the length of the input sequence to the model), 
              and for tgt masking, this must be target sequence length 
        device: (optional) torch.device on which to create the mask

    Returns:
        A Tensor of shape [dim1, dim2]
    """
    mask = torch.triu(torch.ones(dim1, dim2) * float('-inf'), diagonal=1)
    if device is not None:
        mask = mask.to(device)
    return mask



def get_indices_input_target(num_obs, input_len, step_size, forecast_horizon, target_len):
        """
        Produce all the start and end index positions of all sub-sequences.
        The indices will be used to split the data into sub-sequences on which 
        the models will be trained. 

        Returns a tuple with four elements:
        1) The index position of the first element to be included in the input sequence
        2) The index position of the last element to be included in the input sequence
        3) The index position of the first element to be included in the target sequence
        4) The index position of the last element to be included in the target sequence

        
        Args:
            num_obs (int): Number of observations in the entire dataset for which
                            indices must be generated.

            input_len (int): Length of the input sequence (a sub-sequence of 
                             of the entire data sequence)

            step_size (int): Size of each step as the data sequence is traversed.
                             If 1, the first sub-sequence will be indices 0-input_len, 
                             and the next will be 1-input_len.

            forecast_horizon (int): How many index positions is the target away from
                                    the last index position of the input sequence?
                                    If forecast_horizon=1, and the input sequence
                                    is data[0:10], the target will be data[11:taget_len].

            target_len (int): Length of the target / output sequence.
        """

        input_len = round(input_len) # just a precaution
        start_position = 0
        stop_position = num_obs-1 # because of 0 indexing
        
        subseq_first_idx = start_position
        subseq_last_idx = start_position + input_len
        target_first_idx = subseq_last_idx + forecast_horizon
        target_last_idx = target_first_idx + target_len 
        print("target_last_idx is {}".format(target_last_idx))
        print("stop_position is {}".format(stop_position))
        indices = []
        while target_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_last_idx, target_first_idx, target_last_idx))
            subseq_first_idx += step_size
            subseq_last_idx += step_size
            target_first_idx = subseq_last_idx + forecast_horizon
            target_last_idx = target_first_idx + target_len

        return indices

def get_indices_entire_sequence(data: pd.DataFrame, window_size: int, step_size: int) -> list:
        """
        Produce all the start and end index positions that is needed to produce
        the sub-sequences. 

        Returns a list of tuples. Each tuple is (start_idx, end_idx) of a sub-
        sequence. These tuples should be used to slice the dataset into sub-
        sequences. These sub-sequences should then be passed into a function
        that slices them into input and target sequences. 
        
        Args:
            num_obs (int): Number of observations (time steps) in the entire 
                           dataset for which indices must be generated, e.g. 
                           len(data)

            window_size (int): The desired length of each sub-sequence. Should be
                               (input_sequence_length + target_sequence_length)
                               E.g. if you want the model to consider the past 100
                               time steps in order to predict the future 50 
                               time steps, window_size = 100+50 = 150

            step_size (int): Size of each step as the data sequence is traversed 
                             by the moving window.
                             If 1, the first sub-sequence will be [0:window_size], 
                             and the next will be [1:window_size].

        Return:
            indices: a list of tuples
        """

        stop_position = len(data)-1 # 1- because of 0 indexing
        
        # Start the first sub-sequence at index position 0
        subseq_first_idx = 0
        
        subseq_last_idx = window_size
        
        indices = []
        
        while subseq_last_idx <= stop_position:

            indices.append((subseq_first_idx, subseq_last_idx))
            
            subseq_first_idx += step_size
            
            subseq_last_idx += step_size

        return indices


def read_data(data_dir: Union[str, Path] = "data",  
    timestamp_col_name: str="timestamp") -> pd.DataFrame:
    """
    Read data from csv file and return pd.Dataframe object

    Args:

        data_dir: str or Path object specifying the path to the directory 
                  containing the data

        target_col_name: str, the name of the column containing the target variable

        timestamp_col_name: str, the name of the column or named index 
                            containing the timestamps
    """

    # Ensure that `data_dir` is a Path object
    data_dir = Path(data_dir)
    
    # Read csv file
    npy_files = list(data_dir.glob("*.npy"))
    
    if len(npy_files) > 1:
        raise ValueError("data_dir contains more than 1 csv file. Must only contain 1")
    elif len(npy_files) == 0:
        raise ValueError("data_dir must contain at least 1 csv file.")

    data_path = npy_files[0]

    print("Reading file in {}".format(data_path))

    data = np.load(data_path, allow_pickle=True).item()
    data = pd.DataFrame(data)

    # Make sure all "n/e" values have been removed from df. 
    if is_ne_in_df(data):
        raise ValueError("data frame contains 'n/e' values. These must be handled")

    data = to_numeric_and_downcast_data(data)

    # Make sure data is in ascending order by timestamp
    data.sort_values(by=[timestamp_col_name], inplace=True)

    return data

def is_ne_in_df(df:pd.DataFrame):
    """
    Some raw data files contain cells with "n/e". This function checks whether
    any column in a df contains a cell with "n/e". Returns False if no columns
    contain "n/e", True otherwise
    """
    
    for col in df.columns:

        true_bool = (df[col] == "n/e")

        if any(true_bool):
            return True

    return False


def to_numeric_and_downcast_data(df: pd.DataFrame):
    """
    Downcast columns in df to smallest possible version of it's existing data
    type
    """
    fcols = df.select_dtypes('float').columns
    
    icols = df.select_dtypes('integer').columns

    df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
    
    df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')

    return df


# Dataset

In [4]:
class TransformerDataset(Dataset):
    """
    Dataset class used for transformer models.
    
    """
    def __init__(self, 
        data: torch.tensor,
        indices: list, 
        enc_seq_len: int, 
        dec_seq_len: int, 
        target_seq_len: int
        ) -> None:

        """
        Args:

            data: tensor, the entire train, validation or test data sequence 
                        before any slicing. If univariate, data.size() will be 
                        [number of samples, number of variables]
                        where the number of variables will be equal to 1 + the number of
                        exogenous variables. Number of exogenous variables would be 0
                        if univariate.

            indices: a list of tuples. Each tuple has two elements:
                     1) the start index of a sub-sequence
                     2) the end index of a sub-sequence. 
                     The sub-sequence is split into src, trg and trg_y later.  

            enc_seq_len: int, the desired length of the input sequence given to the
                     the first layer of the transformer model.

            target_seq_len: int, the desired length of the target sequence (the output of the model)

            target_idx: The index position of the target variable in data. Data
                        is a 2D tensor
        """
        
        super().__init__()

        self.indices = indices

        self.data = data

        print("From get_src_trg: data size = {}".format(data.size()))

        self.enc_seq_len = enc_seq_len

        self.dec_seq_len = dec_seq_len

        self.target_seq_len = target_seq_len



    def __len__(self):
        
        return len(self.indices)

    def __getitem__(self, index):
        """
        Returns a tuple with 3 elements:
        1) src (the encoder input)
        2) trg (the decoder input)
        3) trg_y (the target)
        """
        # Get the first element of the i'th tuple in the list self.indicesasdfas
        start_idx = self.indices[index][0]

        # Get the second (and last) element of the i'th tuple in the list self.indices
        end_idx = self.indices[index][1]

        sequence = self.data[start_idx:end_idx]

        #print("From __getitem__: sequence length = {}".format(len(sequence)))

        src, trg, trg_y = self.get_src_trg(
            sequence=sequence,
            enc_seq_len=self.enc_seq_len,
            dec_seq_len=self.dec_seq_len,
            target_seq_len=self.target_seq_len
            )

        return src, trg, trg_y
    
    def get_src_trg(
        self,
        sequence: torch.Tensor, 
        enc_seq_len: int, 
        dec_seq_len: int, 
        target_seq_len: int
        ) -> Tuple[torch.tensor, torch.tensor, torch.tensor]:

        """
        Generate the src (encoder input), trg (decoder input) and trg_y (the target)
        sequences from a sequence. 

        Args:

            sequence: tensor, a 1D tensor of length n where 
                    n = encoder input length + target sequence length  

            enc_seq_len: int, the desired length of the input to the transformer encoder

            target_seq_len: int, the desired length of the target sequence (the 
                            one against which the model output is compared)

        Return: 

            src: tensor, 1D, used as input to the transformer model

            trg: tensor, 1D, used as input to the transformer model

            trg_y: tensor, 1D, the target sequence against which the model output
                is compared when computing loss. 
        
        """
        assert len(sequence) == enc_seq_len + target_seq_len, "Sequence length does not equal (input length + target length)"
        
        # encoder input
        src = sequence[:enc_seq_len] 
        
        # decoder input. As per the paper, it must have the same dimension as the 
        # target sequence, and it must contain the last value of src, and all
        # values of trg_y except the last (i.e. it must be shifted right by 1)
        trg = sequence[enc_seq_len-1:len(sequence)-1]
        
        assert len(trg) == target_seq_len, "Length of trg does not match target sequence length"

        # The target sequence against which the model output will be compared to compute loss
        trg_y = sequence[-target_seq_len:]

        assert len(trg_y) == target_seq_len, "Length of trg_y does not match target sequence length"

        return src, trg, trg_y.squeeze(-1) # change size from [batch_size, target_seq_len, num_features] to [batch_size, target_seq_len] 


# Positional Encoder

In [5]:

class PositionalEncoder(nn.Module):
    """
    The authors of the original transformer paper describe very succinctly what 
    the positional encoding layer does and why it is needed:
    
    "Since our model contains no recurrence and no convolution, in order for the 
    model to make use of the order of the sequence, we must inject some 
    information about the relative or absolute position of the tokens in the 
    sequence." (Vaswani et al, 2017)
    Adapted from: 
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(
        self, 
        dropout: float=0.1, 
        max_seq_len: int=5000, 
        d_model: int=512,
        batch_first: bool=False
        ):

        """
        Parameters:
            dropout: the dropout rate
            max_seq_len: the maximum length of the input sequences
            d_model: The dimension of the output of sub-layers in the model 
                     (Vaswani et al, 2017)
        """

        super().__init__()

        self.d_model = d_model
        
        self.dropout = nn.Dropout(p=dropout)

        self.batch_first = batch_first

        # adapted from PyTorch tutorial
        position = torch.arange(max_seq_len).unsqueeze(1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        
        if self.batch_first:
            pe = torch.zeros(1, max_seq_len, d_model)
            
            pe[0, :, 0::2] = torch.sin(position * div_term)
            
            pe[0, :, 1::2] = torch.cos(position * div_term)
        else:
            pe = torch.zeros(max_seq_len, 1, d_model)
        
            pe[:, 0, 0::2] = torch.sin(position * div_term)
        
            pe[:, 0, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, enc_seq_len, dim_val] or 
               [enc_seq_len, batch_size, dim_val]
        """
        if self.batch_first:
            x = x + self.pe[:,:x.size(1)]
        else:
            x = x + self.pe[:x.size(0)]

        return self.dropout(x)


# Inference

In [6]:

def run_encoder_decoder_inference(
    model: nn.Module, 
    src: torch.Tensor, 
    forecast_window: int,
    batch_size: int,
    device,
    batch_first: bool=False
    ) -> torch.Tensor:

    """
    NB! This function is currently only tested on models that work with 
    batch_first = False
    
    This function is for encoder-decoder type models in which the decoder requires
    an input, tgt, which - during training - is the target sequence. During inference,
    the values of tgt are unknown, and the values therefore have to be generated
    iteratively.  
    
    This function returns a prediction of length forecast_window for each batch in src
    
    NB! If you want the inference to be done without gradient calculation, 
    make sure to call this function inside the context manager torch.no_grad like:
    with torch.no_grad:
        run_encoder_decoder_inference()
        
    The context manager is intentionally not called inside this function to make
    it usable in cases where the function is used to compute loss that must be 
    backpropagated during training and gradient calculation hence is required.
    
    If use_predicted_tgt = True:
    To begin with, tgt is equal to the last value of src. Then, the last element
    in the model's prediction is iteratively concatenated with tgt, such that 
    at each step in the for-loop, tgt's size increases by 1. Finally, tgt will
    have the correct length (target sequence length) and the final prediction
    will be produced and returned.
    
    Args:
        model: An encoder-decoder type model where the decoder requires
               target values as input. Should be set to evaluation mode before 
               passed to this function.
               
        src: The input to the model
        
        forecast_horizon: The desired length of the model's output, e.g. 58 if you
                         want to predict the next 58 hours of FCR prices.
                           
        batch_size: batch size
        
        batch_first: If true, the shape of the model input should be 
                     [batch size, input sequence length, number of features].
                     If false, [input sequence length, batch size, number of features]
    
    """

    # Dimension of a batched model input that contains the target sequence values
    target_seq_dim = 0 if batch_first == False else 1

    # Take the last value of thetarget variable in all batches in src and make it tgt
    # as per the Influenza paper
    tgt = src[-1, :, 0] if batch_first == False else src[:, -1, 0] # shape [1, batch_size, 1]

    # Change shape from [batch_size] to [1, batch_size, 1]
    if batch_size == 1 and batch_first == False:
        tgt = tgt.unsqueeze(0).unsqueeze(0) # change from [1] to [1, 1, 1]

    # Change shape from [batch_size] to [1, batch_size, 1]
    if batch_first == False and batch_size > 1:
        tgt = tgt.unsqueeze(0).unsqueeze(-1)

    # Iteratively concatenate tgt with the first element in the prediction
    for _ in range(forecast_window-1):

        # Create masks
        dim_a = tgt.shape[1] if batch_first == True else tgt.shape[0]

        dim_b = src.shape[1] if batch_first == True else src.shape[0]

        tgt_mask = generate_square_subsequent_mask(
            dim1=dim_a,
            dim2=dim_a,
            device=device
            )

        src_mask = generate_square_subsequent_mask(
            dim1=dim_a,
            dim2=dim_b,
            device=device
            )

        # Make prediction
        prediction = model(src, tgt, src_mask, tgt_mask) 

        # If statement simply makes sure that the predicted value is 
        # extracted and reshaped correctly
        if batch_first == False:

            # Obtain the predicted value at t+1 where t is the last time step 
            # represented in tgt
            last_predicted_value = prediction[-1, :, :] 

            # Reshape from [batch_size, 1] --> [1, batch_size, 1]
            last_predicted_value = last_predicted_value.unsqueeze(0)

        else:

            # Obtain predicted value
            last_predicted_value = prediction[:, -1, :]

            # Reshape from [batch_size, 1] --> [batch_size, 1, 1]
            last_predicted_value = last_predicted_value.unsqueeze(-1)

        # Detach the predicted element from the graph and concatenate with 
        # tgt in dimension 1 or 0
        tgt = torch.cat((tgt, last_predicted_value.detach()), target_seq_dim)
    
    # Create masks
    dim_a = tgt.shape[1] if batch_first == True else tgt.shape[0]

    dim_b = src.shape[1] if batch_first == True else src.shape[0]

    tgt_mask = generate_square_subsequent_mask(
        dim1=dim_a,
        dim2=dim_a,
        device=device
        )

    src_mask = generate_square_subsequent_mask(
        dim1=dim_a,
        dim2=dim_b,
        device=device
        )

    # Make final prediction
    final_prediction = model(src, tgt, src_mask, tgt_mask)

    return final_prediction


# Transformer Timeseries

In [7]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, 
                 input_size: int,
                 dec_seq_len: int,
                 batch_first: bool = True,
                 out_seq_len: int = 58,
                 dim_val: int = 512,  
                 n_encoder_layers: int = 4,
                 n_decoder_layers: int = 4,
                 n_heads: int = 8,
                 dropout_encoder: float = 0.2, 
                 dropout_decoder: float = 0.2,
                 dropout_pos_enc: float = 0.1,
                 dim_feedforward_encoder: int = 2048,
                 dim_feedforward_decoder: int = 2048,
                 num_predicted_features: int = 1,
                 verbose: bool = False):

        super().__init__()
        self.dec_seq_len = dec_seq_len
        self.verbose = verbose  # Enable debug printing

        if verbose:
            print(f"Initializing Transformer: input_size={input_size}, num_predicted_features={num_predicted_features}")

        # Input layers
        self.encoder_input_layer = nn.Linear(input_size, dim_val)
        self.decoder_input_layer = nn.Linear(num_predicted_features, dim_val)

        # Positional Encoding
        self.positional_encoding_layer = pe.PositionalEncoder(d_model=dim_val, dropout=dropout_pos_enc)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_val, nhead=n_heads,
                                                   dim_feedforward=dim_feedforward_encoder,
                                                   dropout=dropout_encoder, batch_first=batch_first)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_encoder_layers)

        # Transformer Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=dim_val, nhead=n_heads,
                                                   dim_feedforward=dim_feedforward_decoder,
                                                   dropout=dropout_decoder, batch_first=batch_first)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=n_decoder_layers)

        # Output layer
        self.linear_mapping = nn.Linear(dim_val, num_predicted_features)

    def forward(self, src: torch.Tensor, tgt: torch.Tensor, src_mask: torch.Tensor = None, 
                tgt_mask: torch.Tensor = None) -> torch.Tensor:
        
        # Debug: Print shapes if verbose mode is enabled
        if self.verbose:
            print(f"src shape: {src.shape}, expected last dim: {self.encoder_input_layer.in_features}")
            print(f"tgt shape: {tgt.shape}, expected last dim: {self.decoder_input_layer.in_features}")

        # Validate input dimensions
        assert src.shape[-1] == self.encoder_input_layer.in_features, \
            f"src last dim ({src.shape[-1]}) must match input_size ({self.encoder_input_layer.in_features})"
        assert tgt.shape[-1] == self.decoder_input_layer.in_features, \
            f"tgt last dim ({tgt.shape[-1]}) must match num_predicted_features ({self.decoder_input_layer.in_features})"

        # Encoding
        src = self.encoder_input_layer(src)
        src = self.positional_encoding_layer(src)
        src = self.encoder(src)

        # Decoding
        decoder_output = self.decoder_input_layer(tgt)
        decoder_output = self.decoder(decoder_output, memory=src, tgt_mask=tgt_mask, memory_mask=src_mask)

        # Output mapping
        decoder_output = self.linear_mapping(decoder_output)

        if self.verbose:
            print(f"Output shape: {decoder_output.shape}, expected last dim: {self.linear_mapping.out_features}")

        return decoder_output




# Training

In [9]:
import os
import itertools
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# ----- Hyperparameter Grid -----
# You can adjust these lists.
batch_sizes      = [128]
forecast_windows = [2, 3]
enc_seq_lens     = [6, 7]

# Instead of a separate window_sizes list, compute window_size = enc_seq_len + forecast_window.
hyperparameter_grid = [
    (batch_size, forecast_window, enc_seq_len, enc_seq_len + forecast_window)
    for batch_size in batch_sizes
    for forecast_window in forecast_windows
    for enc_seq_len in enc_seq_lens
]

# ----- Fixed Parameters -----
step_size       = 5
epochs          = 2    # Increase epochs for a more robust comparison if needed
batch_first     = True
target_col_name = 'x'
timestamp_col   = "t"
exogenous_vars  = ['y', 'z']
input_variables = [target_col_name] + exogenous_vars

# Directory containing the training data (.npy files)
data_dir = r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Traindata"

# Device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dictionaries to store results per dataset file
# Each file's results: list of tuples (hyperparams, combined_loss_curve, final_error)
results_by_file = {}

# Also store best hyperparameter combination per file
best_results_by_file = {}

# ----- Print Hyperparameter Combinations -----
print("Hyperparameter combinations:")
for hp in hyperparameter_grid:
    batch_size, forecast_window, enc_seq_len, window_size = hp
    print(f"batch_size={batch_size}, forecast_window={forecast_window}, enc_seq_len={enc_seq_len}, window_size={window_size}")

# ----- Process each .npy file separately -----
for file_name in os.listdir(data_dir):
    if not file_name.endswith('.npy'):
        continue

    file_path = os.path.join(data_dir, file_name)
    data = np.load(file_path)
    print(f"\nProcessing file: {file_name}")
    
    # Get input size based on number of columns in the data
    input_size = data.shape[1]
    
    # Initialize a list to hold loss curves and final errors for each hyperparameter combination
    results_by_file[file_name] = []
    best_error = float('inf')
    best_hyperparams = None
    best_loss_curve = None

    # Iterate through each hyperparameter combination
    for batch_size, forecast_window, enc_seq_len, window_size in hyperparameter_grid:
        print("\n======================================")
        print(f"Training with: batch_size={batch_size}, forecast_window={forecast_window}, enc_seq_len={enc_seq_len}, window_size={window_size}")
        
        # Get training indices for this dataset and hyperparameter setting.
        training_indices = get_indices_entire_sequence(data, window_size, step_size)
        if len(training_indices) == 0:
            print(f"No valid training indices found in {file_name} for window_size={window_size}. Skipping.")
            continue
        
        # Create Dataset and DataLoader.
        training_data = TransformerDataset(
            data=torch.tensor(data).float(),
            indices=training_indices,
            enc_seq_len=enc_seq_len,
            dec_seq_len=forecast_window,
            target_seq_len=forecast_window
        )
        training_dataloader = DataLoader(training_data, batch_size=batch_size, drop_last=True)
        print(f"DataLoader size: {len(training_dataloader)}")
        
        # Initialize the model.
        model = TimeSeriesTransformer(
            input_size=input_size,
            dec_seq_len=forecast_window,
            batch_first=batch_first,
            num_predicted_features=input_size
        ).to(device)
        
        optimizer = torch.optim.Adam(model.parameters())
        criterion = torch.nn.MSELoss()
        
        combined_loss_curve = []  # Stores the loss for every batch during training
        
        # Training loop for this hyperparameter setting.
        for epoch in range(epochs):
            model.train()
            epoch_losses = []
            for src, tgt, tgt_y in training_dataloader:
                src = src.to(device)
                tgt = tgt.to(device)
                tgt_y = tgt_y.to(device)
                
                # Create masks. For the decoder self-attention, the mask is square.
                # (Adjust mask_enc if needed for your model.)
                mask_enc = generate_square_subsequent_mask(forecast_window, enc_seq_len).to(device)
                mask_dec = generate_square_subsequent_mask(forecast_window, forecast_window).to(device)
                
                optimizer.zero_grad()
                prediction = model(src, tgt, mask_enc, mask_dec)
                loss = criterion(tgt_y, prediction)
                loss.backward()
                optimizer.step()
                
                epoch_losses.append(loss.item())
                combined_loss_curve.append(loss.item())
            
            # Print overall progress after each epoch
            avg_epoch_loss = sum(epoch_losses) / len(epoch_losses) if epoch_losses else float('inf')
            print(f"Epoch {epoch+1}/{epochs} completed. Average loss: {avg_epoch_loss:.4f}")
        
        # After training for this hyperparameter setting, use the last batch loss as the representative final error.
        if combined_loss_curve:
            final_error = combined_loss_curve[-1]
            print(f"Final error (last batch loss) for this combination: {final_error:.4f}")
            
            # Store the results for this hyperparameter combination.
            results_by_file[file_name].append(((batch_size, forecast_window, enc_seq_len, window_size), combined_loss_curve, final_error))
            
            # Check if this is the best hyperparameter combination for this file so far.
            if final_error < best_error:
                best_error = final_error
                best_hyperparams = {
                    'batch_size': batch_size,
                    'forecast_window': forecast_window,
                    'enc_seq_len': enc_seq_len,
                    'window_size': window_size
                }
                best_loss_curve = combined_loss_curve
    
    # Save the best results for this file.
    best_results_by_file[file_name] = (best_hyperparams, best_error, best_loss_curve)

# ----- After Training: Print Overview and Plot Subplots -----
print("\n======================================")
print("Final losses for each hyperparameter combination per file:")
for file_name, results in results_by_file.items():
    print(f"\nDataset: {file_name}")
    for hp, _, final_error in results:
        bs, fw, esl, ws = hp
        print(f"  batch_size={bs}, forecast_window={fw}, enc_seq_len={esl}, window_size={ws} -> Final error: {final_error:.4f}")

print("\n======================================")
print("Best hyperparameter combination per dataset:")
for file_name, (best_hyperparams, best_error, _) in best_results_by_file.items():
    print(f"Dataset: {file_name} -> {best_hyperparams} with final error: {best_error:.4f}")

# ----- Plot: Create subplots (one per dataset) and plot all loss curves for that dataset. -----
num_files = len(results_by_file)
fig, axes = plt.subplots(num_files, 1, figsize=(10, 5*num_files))
if num_files == 1:
    axes = [axes]  # ensure axes is iterable

for ax, (file_name, results) in zip(axes, results_by_file.items()):
    ax.set_title(f"Loss Curves for Dataset: {file_name}")
    for hp, loss_curve, _ in results:
        bs, fw, esl, ws = hp
        label = f"bs={bs}, fw={fw}, esl={esl}"
        ax.plot(loss_curve, label=label)
    ax.set_xlabel("Batch Number")
    ax.set_ylabel("Loss")
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


Using device: cpu
Hyperparameter combinations:
batch_size=128, forecast_window=2, enc_seq_len=6, window_size=8
batch_size=128, forecast_window=2, enc_seq_len=7, window_size=9
batch_size=128, forecast_window=3, enc_seq_len=6, window_size=9
batch_size=128, forecast_window=3, enc_seq_len=7, window_size=10

Processing file: lorenz63_on0.05_train.npy

Training with: batch_size=128, forecast_window=2, enc_seq_len=6, window_size=8
From get_src_trg: data size = torch.Size([100000, 3])
DataLoader size: 156


KeyboardInterrupt: 

# implementing the psd

In [ ]:
# write a function that takes the best hyperparameters and trains the model on the entire dataset 
# then, use the trained model to predict a long timeseries of the lorenz system --> many timepoints (100000)
# finally, evaluate the model's performance (generated timeseries) on the test data with the psd metric
# write this psd metric function independently and test it with traindata on testdata


In [ ]:
from psd import power_spectrum_error  # Importing the PSD function

def generate_time_series(model, initial_condition, sequence_length, device):
    """Generate a time series using the trained model."""
    model.eval()
    generated_series = [initial_condition]
    with torch.no_grad():
        for _ in range(sequence_length - 1):
            input_seq = torch.tensor(generated_series[-1]).unsqueeze(0).to(device)
            
            # The target (tgt) is typically the same as the input in autoregressive generation
            tgt_seq = input_seq  # For autoregressive models, the target is usually the input itself
            
            # Pass both input and target to the model
            prediction = model(input_seq, tgt_seq).cpu().numpy()  # Assuming the model expects both src and tgt
            
            generated_series.append(prediction.squeeze(0))
            
    return np.array(generated_series)

def evaluate_power_spectrum(model, train_data, test_data, device):
    """Compute power spectrum distance between generated and test data."""
    test_length = test_data.shape[0]
    
    initial_condition = train_data[np.random.randint(0, len(train_data))]
    generated_series = generate_time_series(model, initial_condition, test_length, device)
    
    pse = power_spectrum_error(generated_series, test_data)
    return pse, generated_series

def process_dataset(train_data, test_data, hyperparameter_grid, load_trained_model, device):
    """Train and evaluate models for a given dataset."""
    print(f"Processing dataset")

    best_pse = float('inf')
    best_hyperparams = None
    best_generated_series = None
    results = {}
    
    for batch_size, forecast_window, enc_seq_len, window_size in hyperparameter_grid:
        print(f"Training with batch_size={batch_size}, forecast_window={forecast_window}, enc_seq_len={enc_seq_len}")
        
        model = load_trained_model().to(device)  # Load trained model
        pse, generated_series = evaluate_power_spectrum(model, train_data, test_data, device)
        results[(batch_size, forecast_window, enc_seq_len)] = pse
        
        if pse < best_pse:
            best_pse = pse
            best_hyperparams = (batch_size, forecast_window, enc_seq_len)
            best_generated_series = generated_series
    
    return results, best_hyperparams, best_generated_series

def train_and_evaluate(hyperparameter_grid, train_data, test_data, load_trained_model, device):
    """Train models and evaluate power spectrum distance for multiple datasets."""
    all_results = {}
    loss_curves = {}

    # Process the dataset with the training and test data provided
    results, best_hyperparams, best_generated_series = process_dataset(
        train_data, test_data, hyperparameter_grid, load_trained_model, device
    )
    all_results["dataset"] = results
    loss_curves["dataset"] = (best_hyperparams, best_generated_series)

    plot_loss_curves(loss_curves)
    print_best_results(all_results, loss_curves)
    return all_results

def plot_loss_curves(loss_curves):
    """Plot the best loss curves for each dataset."""
    fig, axes = plt.subplots(len(loss_curves), 1, figsize=(10, 5 * len(loss_curves)))
    if len(loss_curves) == 1:
        axes = [axes]
    
    for ax, (file_name, (best_hyperparams, generated_series)) in zip(axes, loss_curves.items()):
        ax.set_title(f"Best Loss Curve for {file_name}")
        ax.set_xlabel("Time Step")
        ax.set_ylabel("Generated vs Ground Truth")
        
        batch_size, forecast_window, enc_seq_len = best_hyperparams
        ax.plot(generated_series[:, 0], label=f"bs={batch_size}, fw={forecast_window}, esl={enc_seq_len}")
        
        ax.legend()
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

def print_best_results(all_results, loss_curves):
    """Print best power spectrum distances per dataset."""
    print("\nBest Power Spectrum Distance per dataset:")
    for file_name, (best_hyperparams, _) in loss_curves.items():
        batch_size, forecast_window, enc_seq_len = best_hyperparams
        pse = all_results[file_name][(batch_size, forecast_window, enc_seq_len)]
        print(f"{file_name}, bs={batch_size}, fw={forecast_window}, esl={enc_seq_len}: {pse:.4f}")


# ----- Hyperparameter Grid -----
batch_sizes = [128]
forecast_windows = [2, 3, 4, 5]
enc_seq_lens = [6, 7, 8]
# Compute window_size dynamically
hyperparameter_grid = [
    (batch_size, forecast_window, enc_seq_len, enc_seq_len + forecast_window)
    for batch_size in batch_sizes
    for forecast_window in forecast_windows
    for enc_seq_len in enc_seq_lens
]


# Load the .npy files directly from the path
lorenz63datatrain = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Traindata\lorenz63_on0.05_train.npy")
lorenz96datatrain = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Traindata\lorenz96_on0.05_train.npy")

lorenz63datatest = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Testdata\lorenz63_test.npy")
lorenz96datatest = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Testdata\lorenz96_test.npy")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ----- Train and Evaluate -----
def load_trained_model():
    # This function should load the trained model (e.g., after training or from a saved state)
    # For now, it's a placeholder to return the current model being trained
    return model


# Adjust the code to use the loaded data directly
results = train_and_evaluate(
    hyperparameter_grid=hyperparameter_grid,
    train_data=lorenz63datatrain,  # Training data
    test_data=lorenz63datatest,    # Test data
    load_trained_model=load_trained_model,  # Function to load trained models
    device=device
)

# ----- Plot Results -----
num_files = len(results)
fig, axes = plt.subplots(num_files, 1, figsize=(10, 5 * num_files))
if num_files == 1:
    axes = [axes]

for ax, (file_name, (best_hyperparams, generated_series)) in zip(axes, results.items()):
    ax.set_title(f"Best Loss Curve for {file_name}")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Generated vs Ground Truth")

    batch_size, forecast_window, enc_seq_len = best_hyperparams
    ax.plot(generated_series[:, 0], label=f"bs={batch_size}, fw={forecast_window}, esl={enc_seq_len}")

    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

Using device: cuda
Processing dataset
Training with batch_size=128, forecast_window=2, enc_seq_len=6


AssertionError: src last dim (3) must match input_size (20)

In [ ]:

from psd import power_spectrum_error  # Assuming the psd import is correct

# Load the training and test data for Lorenz 63 and Lorenz 96
lorenz63datatrain = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Traindata\lorenz63_on0.05_train.npy")
lorenz96datatrain = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Traindata\lorenz96_on0.05_train.npy")
lorenz63datatest = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Testdata\lorenz63_test.npy")
lorenz96datatest = np.load(r"C:\Users\hecht\OneDrive - Universität Heidelberg\Dokumente\_Studium\_Master\_2. Semester\DSML\Final Project\DSML_Final_Project\Testdata\lorenz96_test.npy")

# ----- Set Up Hyperparameters -----
batch_sizes      = [128]
forecast_windows = [2]
enc_seq_lens     = [6]
hyperparameter_grid = [
    (batch_size, forecast_window, enc_seq_len, enc_seq_len + forecast_window)
    for batch_size in batch_sizes
    for forecast_window in forecast_windows
    for enc_seq_len in enc_seq_lens
]

# ----- Fixed Parameters -----
step_size       = 5
epochs          = 2
batch_first     = True
target_col_name = 'x'
timestamp_col   = "t"
exogenous_vars  = ['y', 'z']
input_variables = [target_col_name] + exogenous_vars

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

results_by_file = {}
best_results_by_file = {}

# ----- Model Generation -----
def generate_time_series(model, initial_condition, sequence_length, device):
    """Generate a time series using the trained model."""
    model.eval()
    generated_series = [initial_condition]
    with torch.no_grad():
        for _ in range(sequence_length - 1):
            input_seq = torch.tensor(generated_series[-1]).unsqueeze(0).to(device)
            tgt_seq = input_seq  # For autoregressive models, the target is usually the input itself
            prediction = model(input_seq, tgt_seq).cpu().numpy()  # Assuming the model expects both src and tgt
            generated_series.append(prediction.squeeze(0))
            
    return np.array(generated_series)

def evaluate_power_spectrum(model, train_data, test_data, device):
    """Compute power spectrum distance between generated and test data."""
    test_length = test_data.shape[0]  # Length of the test data
    initial_condition = train_data[np.random.randint(0, len(train_data))]  # Random starting point from training data
    generated_series = generate_time_series(model, initial_condition, test_length, device)  # Generate data using the model
    
    # Compute power spectrum error
    pse = power_spectrum_error(generated_series, test_data)  # Compare the generated series with the ground truth (test_data)
    return pse, generated_series

# ----- Dataset and Model Training -----

# Train and evaluate for Lorenz 63 and Lorenz 96 datasets
for data, data_test, file_name in zip(
    [(lorenz63datatrain, lorenz63datatest, "lorenz63")],
    [(lorenz96datatrain, lorenz96datatest, "lorenz96")]
):
    # Use the entire training set and the test set for evaluation
    train_data, test_data = data  # Train data is used for training, test data only for evaluation

    input_size = train_data.shape[1]
    results_by_file[file_name] = []
    best_error = float('inf')
    best_hyperparams = None
    best_loss_curve = None

    for batch_size, forecast_window, enc_seq_len, window_size in hyperparameter_grid:
        print(f"\nTraining with: batch_size={batch_size}, forecast_window={forecast_window}, enc_seq_len={enc_seq_len}, window_size={window_size}")
        
        # Create Dataset and DataLoader
        training_indices = get_indices_entire_sequence(train_data, window_size, step_size)
        if len(training_indices) == 0:
            print(f"No valid training indices found in {file_name} for window_size={window_size}. Skipping.")
            continue

        training_data = TransformerDataset(
            data=torch.tensor(train_data).float(),
            indices=training_indices,
            enc_seq_len=enc_seq_len,
            dec_seq_len=forecast_window,
            target_seq_len=forecast_window
        )
        training_dataloader = DataLoader(training_data, batch_size=batch_size, drop_last=True)

        model = TimeSeriesTransformer(
            input_size=input_size,
            dec_seq_len=forecast_window,
            batch_first=batch_first,
            num_predicted_features=input_size
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters())
        criterion = torch.nn.MSELoss()

        combined_loss_curve = []

        for epoch in range(epochs):
            model.train()
            epoch_losses = []
            for src, tgt, tgt_y in training_dataloader:
                src = src.to(device)
                tgt = tgt.to(device)
                tgt_y = tgt_y.to(device)

                mask_enc = generate_square_subsequent_mask(forecast_window, enc_seq_len).to(device)
                mask_dec = generate_square_subsequent_mask(forecast_window, forecast_window).to(device)

                optimizer.zero_grad()
                prediction = model(src, tgt, mask_enc, mask_dec)
                loss = criterion(tgt_y, prediction)
                loss.backward()
                optimizer.step()

                epoch_losses.append(loss.item())
                combined_loss_curve.append(loss.item())

            avg_epoch_loss = sum(epoch_losses) / len(epoch_losses)
            print(f"Epoch {epoch+1}/{epochs} completed. Average loss: {avg_epoch_loss:.4f}")

        final_error = combined_loss_curve[-1]
        print(f"Final error for this combination: {final_error:.4f}")

        results_by_file[file_name].append(((batch_size, forecast_window, enc_seq_len, window_size), combined_loss_curve, final_error))

        if final_error < best_error:
            best_error = final_error
            best_hyperparams = {
                'batch_size': batch_size,
                'forecast_window': forecast_window,
                'enc_seq_len': enc_seq_len,
                'window_size': window_size
            }
            best_loss_curve = combined_loss_curve

    best_results_by_file[file_name] = (best_hyperparams, best_error, best_loss_curve)

    # Evaluate power spectrum on test data after training
    pse, generated_series = evaluate_power_spectrum(model, train_data, test_data, device)
    print(f"Power Spectrum Error (PSE): {pse:.4f}")

    # Add results to dictionary
    results_by_file[file_name].append((
        best_hyperparams,
        best_error,
        best_loss_curve,
        pse  # Add PSE to the results for further analysis
    ))

# Display the results
print("\n======================================")
for file_name, results in results_by_file.items():
    print(f"Dataset: {file_name}")
    for hp, _, _, pse in results:
        print(f"Best Hyperparameters: {hp} -> Power Spectrum Error (PSE): {pse:.4f}")

Using device: cuda

Processing file: lorenz63_on0.05_train.npy

Training with: batch_size=128, forecast_window=2, enc_seq_len=6, window_size=8
From get_src_trg: data size = torch.Size([100000, 3])
DataLoader size: 156
Epoch 1/2 completed. Average loss: 1.3104
Epoch 2/2 completed. Average loss: 0.9087
Final error (last batch loss) for this combination: 0.7529


AssertionError: For unbatched (2-D) `query`, expected `key` and `value` to be 2-D but found 3-D and 3-D tensors respectively